## 1. importing packages

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

## 2. showcasing the data

In [0]:
df = spark.table("workspace.bronze.prd_info")

In [0]:
df.show(truncate=False)

In [0]:
df.printSchema()

## 3. Define list of Transformations

Transformations:
- extract acutal product key and category id from prd_key
- replace null value in prd_cost with zero
- replace abbreviations in prd_line column
- fix start and end dates

In [0]:
#extract prd_key and cat_id
df = (
  df.withColumn("cat_id", 
    F.regexp_replace(F.substring(F.col("prd_key"), 1, 5), '-', "_")
  )
  .withColumn("prd_key", F.substring(F.col("prd_key"), 7, F.length(F.col("prd_key"))))
)

In [0]:
#replace nulls with zeros in prd_cost column

df = df.withColumn("prd_cost", F.coalesce(F.col("prd_cost"), F.lit(0)))

In [0]:
#replace abbreviations in prd_line column with full names

df = (
    df.withColumn("prd_line",
        F.when(F.upper(F.col("prd_line")) == "R", "Road")
         .when(F.upper(F.col("prd_line")) == "M", "Mountain")
         .when(F.upper(F.col("prd_line")) == "S", "Other Sales")
         .when(F.upper(F.col("prd_line")) == "T", "Touring")
         .otherwise(F.lit("Unknown"))
    )
)

In [0]:
#fix start and end dates

window_spec = Window.partitionBy("prd_key").orderBy("prd_start_dt")

df = df.withColumn("prd_end_dt", 
         (F.lead("prd_start_dt").over(window_spec) - F.lit(1)).cast("date")    
    )

## 4. write to silver table

In [0]:
#renaming columns
_RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}

for old_name, new_name in _RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("silver.crm_prd_info")